In [0]:
CATALOG = "kohei_arai"
SCHEMA = "demo"
TABLE = "calls_log"

# Create widgets for CATALOG and SCHEMA
dbutils.widgets.text("CATALOG", CATALOG)
dbutils.widgets.text("SCHEMA", SCHEMA)
dbutils.widgets.text("TABLE", TABLE)

# Get the widget values
catalog = dbutils.widgets.get("CATALOG")
schema = dbutils.widgets.get("SCHEMA")
table = dbutils.widgets.get("TABLE")

In [0]:
from pyspark.sql.functions import col, lit, rand, floor, when

# Create a DataFrame with 100 records
df = spark.range(100).withColumnRenamed("id", "call_id")

# Define the categories
categories = ["契約内容の確認・変更", "料金や請求に関する確認", "解約や休止の申請・手続き", "個人情報の変更（住所・連絡先など）", "新規申し込みやサービス追加の依頼"]

# Add the category column with random values from the categories list
df = df.withColumn(
    "category",
    when(floor(rand() * len(categories)) == 0, lit(categories[0]))
    .when(floor(rand() * len(categories)) == 1, lit(categories[1]))
    .when(floor(rand() * len(categories)) == 2, lit(categories[2]))
    .when(floor(rand() * len(categories)) == 3, lit(categories[3]))
    .otherwise(lit(categories[4]))
)

# Display the DataFrame
df.createOrReplaceTempView("calls_id")
display(df)

In [0]:
# Create the SQL query using the widget values
query = f"""
CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table}`
SELECT
  call_id,
  ai_query(
    "databricks-llama-4-maverick",
    "貴方はコールセンターの会話に精通するプロフェッショナルです。カテゴリを加味して、コールセンターのやりとりを想定して会話を作成してください。call_idが違う場合は違う会話を想定してください。Call ID:" || call_id || "Category:" || category
  ) AS content
FROM calls_id
"""

# Run the SQL query
spark.sql(query)